## **FINAL SUBMISSION**

In [1]:
# I will use 2 notebook for submission

# NOTEBOOK 1

In [2]:
# !pip install -q /kaggle/input/tabpfn-019-whl/tabpfn-0.1.9-py3-none-any.whl
# !mkdir /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
# !cp /kaggle/input/tabpfn-019-whl/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pickle
import os
from copy import deepcopy
from functools import partial
import random
import gc
# from tabpfn import TabPFNClassifier
# Import sklearn classes for model selection, cross validation, and performance evaluation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder
from imblearn.under_sampling import RandomUnderSampler

# Import libraries for Hypertuning
import optuna

# Import libraries for gradient boosting
import xgboost as xgb
import lightgbm as lgb

import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from catboost import Pool

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Prepare train and test sets

In [4]:
filepath = '/kaggle/input/icr-identify-age-related-conditions'
df_train = pd.read_csv(os.path.join(filepath, 'train.csv'), index_col=[0])
df_test = pd.read_csv(os.path.join(filepath, 'test.csv'), index_col=[0])
greeks = pd.read_csv(f'{filepath}/greeks.csv')


df_train['EJ'] = df_train['EJ'].replace({'A': 0, 'B': 1})
df_test['EJ']  = df_test['EJ'].replace({'A': 0, 'B': 1})
df_train = df_train.rename(columns={'BD ': 'BD', 'CD ': 'CD', 'CW ': 'CW', 'FD ': 'FD'})
df_test = df_test.rename(columns={'BD ': 'BD', 'CD ': 'CD', 'CW ': 'CW', 'FD ': 'FD'})
target_col = 'Class'

X_train = df_train.drop([f'{target_col}'],axis=1).reset_index(drop=True)
y_train = df_train[f'{target_col}'].reset_index(drop=True)
X_test = df_test.reset_index(drop=True)

# for data in [X_train, X_test]:
#     data['AB/DA'] = data['AB'] / data['DA']
#     data['BP*DU'] =  data['BP'] * data['DU']
#     data['DF*GL_2'] = data['DF'] * data['GL'].pow(2)
#     data['DI/DL'] = data['DI'] / data['DL']
#     data['CD/GE'] = data['CD'] / data['GE']
#     data['EE*EP_2'] = data['EE'] * data['EP'].pow(2)
#     data['AZ/EP'] = data['AZ'] / data['EP']
#     data['CD/FD'] = data['CD'] / data['FD']
#     data['FR/GE'] = data['FR'] / data['GE']
#     data['AF/DL'] = data['AF'] / data['DL']
#     data['DA_2*GF'] =  data['DA'].pow(2) * data['GF']
#     data['AB/DE']  = data['AB'] / data['DE']
#     data['DU*GH'] = data['DU'] * data['GH']
#     data['BQ/GF'] = data['BQ'] / data['GF']
#     data['DF*DI_2'] = data['DF'] * data['DI'].pow(2)
#     data[ 'FC/FL'] = data['FC'] / data['FL']
#     data['CC/DI'] = data['CC'] / data['DI']
#     data['DU*FL_2'] = data['DU'] * data['FL'].pow(2)
#     data['AF/EG'] = data['AF'] / data['EG']
#     data['CC_2*DN'] = data['CC'].pow(2) * data['DN']
#     data['AF/AY'] = data['AF'] / data['AY']
#     data['BN/DI'] = data['BN'] / data['DI']
#     data['DI_2*DU'] = data['DI'].pow(2) * data['DU']
#     data['DF*GL'] = data['DF'] * data['GL']
#     data['DL_2*EE'] = data['DL'].pow(2) * data['EE']
#     data['AB_2*BQ'] = data['AB'].pow(2) * data['BQ']
#     data['DI*DU'] = data['DI'] * data['DU']
#     data['BQ*DY'] = data['BQ'] * data['DY']
#     data['CR_2*DH'] = data['CR'].pow(2) * data['DH']
#     data['BQ*DY_2'] = data['BQ'] * data['DY'].pow(2)
#     data['AF/EP'] = data ['AF'] / data['EP']
#     data['AB/EG'] = data['AB'] / data['EG']
#     data['AB*EL'] = data['AB'] * data['EL']
#     data['AY/DU'] = data['AY'] / data['DU']
#     data['CC/DU'] = data['CC'] / data['DU']
#     data['CR/DU'] = data['CR'] / data['DU']
#     data['DL/DY'] = data['DL'] / data['DY']
#     data['DA_2*EE'] = data['DA'].pow(2) * data['EE']
#     data['AB/FD'] = data['AB'] / data['FD']
#     data['DA_2*DE'] = data['DA'].pow(2) * data['DE']
#     data['DU/GL'] = data['DU'] / data['GL']
#     data['CD/EE'] = data['CD'] / data['EE']
#     data['BQ_2*FE'] = data['BQ'].pow(2) * data['FE']
#     data['AF/GE'] = data['AF'] / data['GE']
#     data['AB/CR'] = data['AB'] / data['CR']
#     data['DU/FI'] = data['DU'] / data['FI']
#     data['BC*FL_2'] = data['BC'].pow(2) * data['FL'].pow(2)
#     data['CC*CR_2'] = data['CC'] * data['CR'].pow(2)
#     data['AB/DU'] = data['AB'] / data['DU']
#     data['AB/FL'] = data['AB'] / data['FL']
#     data['BC*DU_2'] = data['BC'] * data['DU'].pow(2)
#     data['DU_2*FR'] = data['DU'].pow(2) * data['FR']
#     data['DH/DU'] = data['DH'] / data['DU']
#     data['DA/DI'] = data['DA'] / data['DI']
#     data['AB/CH'] = data['AB'] / data['CH']
#     data['BQ/EE'] = data['BQ'] / data['EE']
#     data['AM/DU'] = data['AM'] / data['DU']
    
drop_cols = ['AH', 'AR', 'AY', 'BZ', 'CF', 'CL', 'GI', 'DH', 'FD', 'AX', 'BC']
X_train = X_train.drop(drop_cols, axis=1)
X_test =  X_test.drop(drop_cols, axis=1)

X_train.head(3)

,AB,AF,AM,AZ,BD,BN,BP,BQ,BR,CB,...,FE,FI,FL,FR,FS,GB,GE,GF,GH,GL
0,0.209377,3109.03329,22.394407,9.812214,4126.58731,22.5984,175.638726,152.707705,823.928241,47.223358,...,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,0.120343
1,0.145282,978.76416,36.968889,13.517790,5496.92824,19.4205,155.868030,14.754720,51.216883,30.284345,...,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,21.978000
2,0.470030,2635.10654,32.360553,12.824570,5135.78024,26.4825,128.988531,219.320160,482.141594,32.563713,...,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,0.196941


In [5]:
log_cols = [_ for _ in X_train.columns if _ not in ['EJ', 'BN', 'CW', 'EL', 'GL']]
X_train.loc[:, log_cols] = np.log1p(X_train.loc[:, log_cols])
X_test.loc[:, log_cols] = np.log1p(X_test.loc[:, log_cols])
X_train.head(3)

,AB,AF,AM,AZ,BD,BN,BP,BQ,BR,CB,...,FE,FI,FL,FR,FS,GB,GE,GF,GH,GL
0,0.190105,8.042389,3.152497,2.380676,8.325448,22.5984,5.174107,5.035053,6.715296,3.875844,...,9.108229,1.522452,2.116034,1.007429,0.090592,2.512776,4.298795,7.603305,3.141400,0.120343
1,0.135651,6.887312,3.636767,2.675375,8.612127,19.4205,5.055405,2.757140,3.955406,3.443118,...,8.822617,2.430004,0.159760,0.403503,0.450395,2.331435,4.298795,10.239337,3.405702,21.978000
2,0.385283,7.877058,3.507374,2.626447,8.544182,26.4825,4.867446,5.395082,6.180310,3.513446,...,9.028807,2.535831,2.164421,0.680852,0.787921,3.639631,4.495461,9.523541,3.368083,0.196941


In [6]:
#  selected = ['AB/DA', 'BP*DU', 'DF*GL_2',
#        'DI/DL', 'CD/GE', 'EE*EP_2', 'AZ/EP', 'CD/FD', 'FR/GE', 'AF/DL',
#        'DA_2*GF', 'AB/DE', 'DU*GH', 'BQ/GF', 'DF*DI_2', 'FC/FL', 'CC/DI',
#        'DU*FL_2', 'AF/EG', 'CC_2*DN', 'AF/AY', 'BN/DI', 'DI_2*DU', 'DF*GL',
#        'DL_2*EE', 'AB_2*BQ', 'DI*DU', 'BQ*DY', 'CR_2*DH', 'BQ*DY_2', 'AF/EP',
#        'AB/EG', 'AB*EL', 'AY/DU', 'CC/DU', 'CR/DU', 'DA_2*EE', 'AB/FD',
#        'DA_2*DE', 'DU/GL', 'DL/DY', 'CD/EE', 'BQ_2*FE', 'AF/GE', 'AB/CR',
#        'DU/FI', 'BC*FL_2', 'CC*CR_2', 'AB/DU', 'AB/FL', 'BC*DU_2', 'DU_2*FR',
#        'DH/DU', 'DA/DI', 'AB/CH', 'BQ/EE', 'AM/DU']

# X_train = X_train[selected]
# X_test = X_test[selected]

In [7]:
numeric_columns = [_ for _ in X_train.columns if _ not in ['EJ']]
sc = StandardScaler() # MinMaxScaler or StandardScaler
X_train[numeric_columns] = sc.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = sc.transform(X_test[numeric_columns])

print(f"X_train shape :{X_train.shape} , y_train shape :{y_train.shape}")
print(f"X_test shape :{X_test.shape}")

scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
feat = X_train.columns
# Delete the train and test dataframes to free up memory
del df_train, df_test

X_train.head(3)

X_train shape :(617, 45) , y_train shape :(617,)
X_test shape :(5, 45)


,AB,AF,AM,AZ,BD,BN,BP,BQ,BR,CB,...,FE,FI,FL,FR,FS,GB,GE,GF,GH,GL
0,-0.793051,0.110642,-0.067111,0.025434,-0.598002,0.339209,-0.334638,0.868474,0.332908,0.046309,...,0.207783,-3.120884,0.843092,0.506545,-0.737475,-1.149987,-0.627728,-0.819140,-0.956392,-0.815091
1,-1.045391,-1.520469,0.506365,0.735599,0.281949,-0.575174,-0.605475,-1.238041,-2.518775,-0.446284,...,-0.228882,0.209787,-1.061092,-0.798943,0.593821,-1.585989,-0.627728,0.974516,-0.095903,1.303181
2,0.111391,-0.122825,0.353137,0.617694,0.073394,1.456789,-1.034331,1.201413,-0.219872,-0.366226,...,0.086357,0.598166,0.890191,-0.199406,1.842690,1.559328,-0.281317,0.487461,-0.218376,-0.807668


## Define Model

In [8]:
class Splitter:
    def __init__(self, kfold=True, n_splits=10, greeks=pd.DataFrame()):
        self.n_splits = n_splits
        self.kfold = kfold
        self.greeks = greeks

    def split_data(self, X, y, random_state_list):
        if self.kfold == 'skf':
            for random_state in random_state_list:
                kf = StratifiedKFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
                for train_index, val_index in kf.split(X, y):
                    if type(X) is np.ndarray:
                        # DOWNSAMPLE NOTE WORK
                        y_train = y.loc[train_index]
                        UNT = y_train.loc[y_train==0].sample(
                            frac=0.6, random_state=bag*n_reapts+n_splits).index.values
                        train_index = np.setdiff1d(train_index,UNT)
                        
                        X_train, X_val = X[train_index], X[val_index]
                        y_train, y_val = y[train_index], y[val_index]
                          
                    else:
                        y_train = y.loc[train_index]
                        UNT = y_train.loc[y_train==0].sample(
                            frac=0.6, random_state=bag*n_reapts+n_splits).index.values
                        train_index = np.setdiff1d(train_index,UNT)
                        
                        # DOWNSAMPLE NOTE WORK
                        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                    yield X_train, X_val, y_train, y_val
        else:
            raise ValueError(f"Invalid kfold: Must be True")

class Classifier:
    def __init__(self, n_estimators=100, device="cpu", random_state=0):
        self.n_estimators = n_estimators
        self.device = device
        self.random_state = random_state
        self.models = self._define_model()
        self.models_name = list(self._define_model().keys())
        self.len_models = len(self.models)
        
    def _define_model(self):
        
        xgb_params = {
            'n_estimators': self.n_estimators,
            'learning_rate': 0.413327571405248,
            'booster': 'gbtree',
            'lambda': 0.0000263894617720096,
            'alpha': 0.000463768723479341,
            'subsample': 0.237467672874133,
            'colsample_bytree': 0.618829300507829,
            'max_depth': 5,
            'min_child_weight': 9,
            'eta': 2.09477807126539E-06,
            'gamma': 0.000847289463422307,
            'grow_policy': 'depthwise',
            'n_jobs': -1,
            'objective': 'binary:logistic',
#             'eval_metric': 'logloss',
            'verbosity': 0,
            'tree_method':'gpu_hist',
            'random_state': self.random_state,
        }
        xgb_params1 = {
            'n_estimators': 900,
            'learning_rate': 0.09641232707445854,
            'booster': 'gbtree',
            'lambda': 4.666002223704784,
            'alpha': 3.708175990751336,
            'subsample': 0.6100174145229473,
            'colsample_bytree': 0.5506821152321051,
            'max_depth': 7,
            'min_child_weight': 3,
            'eta': 1.740374368661041,
            'gamma': 0.007427363662926455,
            'grow_policy': 'depthwise',
            'objective': 'binary:logistic',
#             'eval_metric': 'logloss',
            'verbosity': 0,
            'random_state': self.random_state,
            'scale_pos_weight': scale_pos_weight
        }
        
        xgb_params2 = {
            'n_estimators': 650,
            'learning_rate': 0.012208383405206188,
            'booster': 'gbtree',
            'lambda': 0.009968756668882757,
            'alpha': 0.02666266827121168,
            'subsample': 0.7097814108897231,
            'colsample_bytree': 0.7946945784285216,
            'max_depth': 3,
            'min_child_weight': 4,
            'eta': 0.5480204506554545,
            'gamma': 0.8788654128774149,
            'scale_pos_weight': 4.71,
            'objective': 'binary:logistic',
#             'eval_metric': 'logloss',
            'verbosity': 0,
            'random_state': self.random_state,
            'scale_pos_weight': scale_pos_weight
        }
        
        xgb_params3 = {
            'colsample_bytree': 0.5646751146007976,
            'gamma': 7.788727238356553e-06,
            'learning_rate': 0.1419865761603358,
            'max_bin': 824,
            'min_child_weight': 1,
            'random_state': 811996,
            'reg_alpha': 1.6259583347890365e-07,
            'reg_lambda': 2.110691851528507e-08,
            'subsample': 0.879020578464637,
            'objective': 'binary:logistic',
#             'eval_metric': 'logloss',
            'max_depth': 3,
            'n_jobs': -1,
            'verbosity': 0,
            'random_state': self.random_state,
            'scale_pos_weight': scale_pos_weight
        }
        
        xgb_params4 = {
            'random_state': self.random_state,
            'colsample_bytree': 0.4836462317215041,
            'eta': 0.05976752607337169,
            'gamma': 1,
            'lambda': 0.2976432557733288,
            'max_depth': 6,
            'min_child_weight': 1,
            'n_estimators': 550,
            'objective': 'binary:logistic',
            'scale_pos_weight': 4.260162886376033,
            'subsample': 0.7119282378433924,
        }
        
        xgb_params5 = {
            'colsample_bytree': 0.8757972257439255,
            'gamma': 0.11135738771999848,
            'max_depth': 7,
            'min_child_weight': 3,
            'reg_alpha': 0.4833998914998038,
            'reg_lambda': 0.006223568555619563,
            'scale_pos_weight': 8,
            'subsample': 0.7056434340275685,
            'random_state': self.random_state
        }
        
        xgb_params6 = {
            'max_depth': 5, 
            'min_child_weight': 2.934487833919741,
            'learning_rate': 0.11341944575807082, 
            'subsample': 0.9045063514419968,
            'gamma': 0.4329153382843715,
            'colsample_bytree': 0.38872702868412506,
            'colsample_bylevel': 0.8321880031718571,
            'colsample_bynode': 0.802355707802605,
            'random_state': self.random_state
       }
        
        xgb_params7 = { 'colsample_bylevel' : 0.6, 'colsample_bytree' : 1.0, 
                        'learning_rate' : 0.3,  'max_depth' : 4,
                       'subsample' : 1.0,    'scale_pos_weight' : 5,
                       'random_state': self.random_state}
        
        xgb_params8 = {
            "max_depth": 2,
            "n_estimators": 200,
            "learning_rate": 0.4,
            "min_child_weight": 0.1,
            "max_delta_step": 0.35,
            "colsample_bytree": 0.3,
            "colsample_bylevel": 0.7,
            "gamma": 1e-4,
            "reg_alpha": 2e-3,
            'random_state': self.random_state
        } 
        if self.device == 'gpu':
            xgb_params['tree_method'] = 'gpu_hist'
            xgb_params['predictor'] = 'gpu_predictor'
            
            xgb_params1['tree_method'] = 'gpu_hist'
            xgb_params1['predictor'] = 'gpu_predictor'
            
            xgb_params2['tree_method'] = 'gpu_hist'
            xgb_params2['predictor'] = 'gpu_predictor'
            
            xgb_params3['tree_method'] = 'gpu_hist'
            xgb_params3['predictor'] = 'gpu_predictor'
            
            xgb_params4['tree_method'] = 'gpu_hist'
            xgb_params4['predictor'] = 'gpu_predictor'
            
            xgb_params5['tree_method'] = 'gpu_hist'
            xgb_params5['predictor'] = 'gpu_predictor'
            
            xgb_params6['tree_method'] = 'gpu_hist'
            xgb_params6['predictor'] = 'gpu_predictor'
    
            xgb_params7['tree_method'] = 'gpu_hist'
            xgb_params7['predictor'] = 'gpu_predictor'
            
            xgb_params8['tree_method'] = 'gpu_hist'
            xgb_params8['predictor'] = 'gpu_predictor'
            
        lgb_params = {
            'n_estimators': self.n_estimators,
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'learning_rate': 0.005,
            'num_leaves': 5,
            'colsample_bytree': 0.50,
            'subsample': 0.80,
            'reg_alpha': 2, 
            'reg_lambda': 4,
            'n_jobs': -1,
            'is_unbalance':True,
            'device': self.device,
            'random_state': self.random_state
        }

        
        lgb1_params = {
            'n_estimators': self.n_estimators,
            'learning_rate': 0.190197487721534,
            'reg_alpha': 0.00749112221417973,
            'reg_lambda': 0.000548118227209224,
            'num_leaves': 17,
            'colsample_bytree': 0.547257860506146,
            'subsample': 0.592628085686409,
            'subsample_freq': 2,
            'min_child_samples': 64,
            'objective': 'binary',
            #'metric': 'binary_error',
            'boosting_type': 'gbdt',
            'is_unbalance':True,
            'device': self.device,
            'random_state': self.random_state
        } 
        
        lgb2_params = {
            'n_estimators': self.n_estimators,
            'learning_rate': 0.181326407627473,
            'reg_alpha': 0.000030864084239014,
            'reg_lambda': 0.0000395714763869486,
            'num_leaves': 122,
            'colsample_bytree': 0.75076596295323,
            'subsample': 0.6303245788342,
            'subsample_freq': 3,
            'min_child_samples': 72,
            'objective': 'binary',
            #'metric': 'binary_error',
            'boosting_type': 'gbdt',
            'is_unbalance':True,
            'device': self.device,
            'random_state': self.random_state
        }
        
        lgb3_params = {
            "max_depth": 4,
            "num_leaves": 9,
            "min_child_samples": 17,
            "n_estimators": 200,
            "learning_rate": 0.15,
            "colsample_bytree": 0.4,
            "min_split_gain": 1e-4,
            "reg_alpha": 1e-2,
            "reg_lambda": 5e-3,
            'device': self.device,
            'random_state': self.random_state
        }
        
        lgb4_params = {
            "max_depth": 4,
            "num_leaves": 9,
            "min_child_samples": 20,
            "n_estimators": 200,
            "learning_rate": 0.15,
            "colsample_bytree": 0.4,
            "reg_alpha": 1e-2,
            "min_split_gain": 1e-4
            , 'device': self.device
            ,'random_state': self.random_state
        }
        
        lgb5_params = {"boosting_type": 'goss'
           , "learning_rate": 0.06733232950390658
           , "n_estimators": 50000
           , "subsample": 0.6970532011679706
           , "colsample_bytree": 0.6055755840633003
           , "class_weight": 'balanced'
#            , "metric": 'none'
           , "is_unbalance": True
           , "max_depth": 8
           , 'device': self.device
           ,'random_state': self.random_state}
        
        cat_params = {
            'iterations': self.n_estimators,
            'colsample_bylevel': 0.0513276895988184,
            'depth': 2,
            'learning_rate': 0.0256579773375401,
            'l2_leaf_reg': 8.22319805476255,
            'random_strength': 0.11327724457066,
            'od_type': "Iter", 
            'od_wait': 72,
            'bootstrap_type': "Bayesian",
            'grow_policy': 'SymmetricTree',
            'bagging_temperature': 9.58737431845122,
            'auto_class_weights': 'Balanced',
#             'task_type': self.device.upper(),
            'random_state': self.random_state
        }

        
        cat_params1 = {
            "depth":4,
            "l2_leaf_reg" : 50,
            "learning_rate" : 0.3,
            "objective": "Logloss", 
            "logging_level":"Silent", 
            "iterations" : 1000,
            "auto_class_weights" : "Balanced",
            "one_hot_max_size":10,
            'random_state': self.random_state,  }
        
        cat_params2 = {
                "objective": "Logloss",
                'bagging_temperature' : 1.0, 
                'boosting_type' : 'Plain', 
                'depth' : 5,
                'grow_policy' : 'Depthwise',
                "iterations" : 100,
                'l2_leaf_reg' : 2, 
                'learning_rate' : 0.07, 
                'max_bin' : 65535, 'max_leaves' : 31,
                'min_data_in_leaf' : 3,
                'random_strength' : 5.0, 
                'rsm' : 0.5,
                'sampling_frequency' : 'PerTreeLevel',
                'scale_pos_weight' : 5.0,
                'thread_count' : -1,
#                 'task_type': self.device.upper(),
                'random_state': self.random_state
        }
        
        cat_params3 = {
#             "objective": "Logloss",
            'learning_rate': 0.05, 
            "iterations" : 250,
            'depth': 3, # 
            'colsample_bylevel': 0.50,
            'subsample': 0.80,
            'l2_leaf_reg': 3, # 3, 30
            'auto_class_weights': 'Balanced',
#             'task_type': self.device.upper(),
            'random_state': self.random_state
        }
        
        cat_params4 = {
            "learning_rate": 0.003,
            "depth": 4,
            "iterations" : 1000,
            'auto_class_weights':'Balanced',
            'loss_function':'MultiClass',
            'eval_metric':'MultiClass:use_weights=False',
#             'task_type': self.device.upper(),
            'random_state': self.random_state
    }
        
            
        models = {
            'xgb' : xgb.XGBClassifier(**xgb_params),
            'xgb1': xgb.XGBClassifier(**xgb_params1),
            'xgb2': xgb.XGBClassifier(**xgb_params2),
            'xgb3': xgb.XGBClassifier(**xgb_params3),
            'xgb4': xgb.XGBClassifier(**xgb_params4),
            'xgb5': xgb.XGBClassifier(**xgb_params5),
            'xgb6': xgb.XGBClassifier(**xgb_params6),
            'xgb7': xgb.XGBClassifier(**xgb_params7),
            'xgb8': xgb.XGBClassifier(**xgb_params8),
            
            'lgb' : lgb.LGBMClassifier(**lgb_params),
            'lgb1': lgb.LGBMClassifier(**lgb1_params),
            'lgb2': lgb.LGBMClassifier(**lgb2_params),
            'lgb3': lgb.LGBMClassifier(**lgb3_params),
            'lgb4': lgb.LGBMClassifier(**lgb4_params),
            'lgb5': lgb.LGBMClassifier(**lgb5_params),
#             'cat': CatBoostClassifier(**cat_params),
#             'cat1': CatBoostClassifier(**cat_params1),
#             'cat2': CatBoostClassifier(**cat_params2),
#             'cat3': CatBoostClassifier(**cat_params3),
#             'cat4': CatBoostClassifier(**cat_params4),
            
        }
        
        return models

## Metrics

In [9]:
def calc_log_loss_weight(y_true):
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    return w0, w1

# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/415000
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import log_loss
# def balanced_log_loss(y_true, y_pred):
#     sw = compute_sample_weight(class_weight='balanced', y=y_true)
#     balanced_log_loss = log_loss(y_true, y_pred, eps=1e-15, sample_weight=sw)
#     return balanced_log_loss


def balanced_log_loss(y_true, y_pred):
    y_pred = np.stack([1-y_pred,y_pred]).T
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    y_pred / np.sum(y_pred, axis=1)[:, None]
    nc = np.bincount(y_true)
    
    logloss = (-1/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(y_pred[:,0]))) - 1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred[:,1])))) / 2
    return logloss

## Optimizer (--> Optimize balanced_log_loss)

In [10]:
class OptunaWeights:
    def __init__(self, random_state, n_trials = 3000):
        self.study = None
        self.weights = None
        self.random_state = random_state
        self.n_trials = n_trials

    def _objective(self, trial, y_true, y_preds):
        # Define the weights for the predictions from each model
        weights = [trial.suggest_float(f"weight{n}", 1e-15, 1) for n in range(len(y_preds))]

#         weights = [trial.suggest_float(f"weight{n}", 0.0, 1.0, step=0.1) for n in range(len(y_preds))]
        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=weights)

        # Calculate the score for the weighted prediction
        # score = log_loss(y_true, weighted_pred)
        score = balanced_log_loss(y_true, weighted_pred)
        return score

    def fit(self, y_true, y_preds):
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = optuna.samplers.CmaEsSampler(seed=self.random_state)
        pruner = optuna.pruners.HyperbandPruner()
        self.study = optuna.create_study(sampler=sampler, pruner=pruner, study_name="OptunaWeights", direction='minimize')
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        self.study.optimize(objective_partial, n_trials=self.n_trials)
        self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]

    def predict(self, y_preds):
        assert self.weights is not None, 'OptunaWeights error, must be fitted before predict'
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=self.weights)
        return weighted_pred

    def fit_predict(self, y_true, y_preds):
        self.fit(y_true, y_preds)
        return self.predict(y_preds)
    
    def weights(self):
        return self.weights

## Base Model

In [11]:
%%time

kfold = 'skf'
n_splits = 10
n_reapts = 20
random_state = 42
n_estimators = 99999
early_stopping_rounds = 200
device = 'gpu'

# Fix seed
random.seed(random_state)
random_state_list = random.sample(range(9999), n_reapts)
bag = 1
# Initialize an array for storing test predictions
classifier = Classifier(n_estimators, device, random_state)
test_predss = np.zeros((X_test.shape[0]))
oof_predss = np.zeros((X_train.shape[0], n_reapts))

ensemble_score, ensemble_score_ = [], []
weights = []
oof_each_predss = []

oof_each_preds = np.zeros((X_train.shape[0], classifier.len_models))
test_each_predss = []
test_each_preds = np.zeros((X_test.shape[0], classifier.len_models))
trained_models = {'xgb':[], 'cat':[]}
score_dict = dict(zip(classifier.models_name, [[] for _ in range(classifier.len_models)]))

splitter = Splitter(kfold=kfold, n_splits=n_splits, greeks=greeks.iloc[:,1:-1])
for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, y_train, random_state_list=random_state_list)):
    
    n = i % n_splits
    m = i // n_splits  # to handled FOLD
            
    # Get a set of classifier models
    classifier = Classifier(n_estimators, device, random_state_list[m])
    models = classifier.models
    
    # Initialize lists to store oof and test predictions for each base model
    oof_preds = []
    test_preds = []
    if n == 0: print('\n#### Bag/SEED no.', bag); bag+=1
    print('# FOLD',n+1, ' ', end = '')
    # Loop over each base model and fit it to the training data, evaluate on validation data, and store predictions
    for name, model in models.items():
        if ('xgb' in name) or ('lgb' in name) or ('cat' in name):
            # weight of sample
            train_w0, train_w1 = calc_log_loss_weight(y_train_)
            valid_w0, valid_w1 = calc_log_loss_weight(y_val)
            w = y_train_.map({0: train_w0, 1: train_w1})
            e = y_val.map({0: valid_w0, 1: valid_w1})
                             
            if 'xgb' in name:
                model.fit(
                    X_train_, y_train_, sample_weight  = w , 
                     eval_metric = ["logloss"],
                    eval_set = [(X_val, y_val)], sample_weight_eval_set = [e] ,
                    early_stopping_rounds = 200,
                    verbose=0)
            elif 'lgb' in name:
                model.fit(
                    X_train_, y_train_, sample_weight = w , 
                    eval_metric = 'logloss',
                    eval_set = [(X_val, y_val)], eval_sample_weight = [e],
                    early_stopping_rounds = 1000, verbose = 0)
                
            elif 'cat' in name:
                model.fit(
                    Pool(X_train_, y_train_, weight = w) , 
                    eval_set = Pool(X_val, y_val, weight = e),
#                     early_stopping_rounds = 10, 
                    verbose = 0)
        else:
            model.fit(X_train_ , y_train_)
            
        if name in trained_models.keys():
            trained_models[f'{name}'].append(deepcopy(model))
        
        test_pred = model.predict_proba(X_test)[:, 1].reshape(-1)
        y_val_pred = model.predict_proba(X_val)[:, 1].reshape(-1)
        
        score = balanced_log_loss(y_val, y_val_pred)
        score_dict[name].append(score)
#         print('=> Fold',n,', ',end='')
        print(f'({name} {score:.3f}) ' , end='')
#         print(f'{score:.3f}', end = '')
#         print(f'{name} [FOLD-{n} SEED-{random_state_list[m]}] BalancedLogLoss score: {score:.5f}')
        
        oof_preds.append(y_val_pred)
        test_preds.append(test_pred)
    
    # Use Optuna to find the best ensemble weights
    optweights = OptunaWeights(random_state=random_state_list[m])
    y_val_pred = optweights.fit_predict(y_val.values, oof_preds)
    
    score = balanced_log_loss(y_val, y_val_pred)
    score_ = roc_auc_score(y_val, y_val_pred)
    print(f'\n==> Ensemble Score {score:.5f}')
    ensemble_score.append(score)
    ensemble_score_.append(score_)
    weights.append(optweights.weights)
    
    # Predict to X_test by the best ensemble weights
    test_predss += optweights.predict(test_preds) / (n_splits * len(random_state_list))
    
    oof_predss[X_val.index, m] += optweights.predict(oof_preds)
    oof_each_preds[X_val.index] = np.stack(oof_preds).T
    test_each_preds += np.array(test_preds).T / n_splits
    
    # After End of FOLD LOOP in One Bag
    if n == (n_splits - 1):
        oof_each_predss.append(oof_each_preds)
        oof_each_preds = np.zeros((X_train.shape[0], classifier.len_models))
        test_each_predss.append(test_each_preds)
        test_each_preds = np.zeros((X_test.shape[0], classifier.len_models))
    gc.collect()

oof_each_predss = np.mean(np.array(oof_each_predss), axis=0)

test_each_predss = np.mean(np.array(test_each_predss), axis=0)
oof_each_predss = np.concatenate([oof_each_predss, np.mean(oof_predss, axis=1).reshape(-1, 1)], axis=1)
test_each_predss = np.concatenate([test_each_predss, test_predss.reshape(-1, 1)], axis=1)
print()


#### Bag/SEED no. 1
# FOLD 1  (xgb 0.401) (xgb1 0.286) (xgb2 0.301) (xgb3 0.353) (xgb4 0.293) (xgb5 0.342) (xgb6 0.334) (xgb7 0.370) (xgb8 0.342) (lgb 0.296) (lgb1 0.303) (lgb2 0.283) (lgb3 0.202) (lgb4 0.193) (lgb5 0.306) 
==> Ensemble Score 0.18656
# FOLD 2  (xgb 0.273) (xgb1 0.172) (xgb2 0.167) (xgb3 0.116) (xgb4 0.115) (xgb5 0.176) (xgb6 0.169) (xgb7 0.070) (xgb8 0.124) (lgb 0.136) (lgb1 0.204) (lgb2 0.212) (lgb3 0.136) (lgb4 0.152) (lgb5 0.178) 
==> Ensemble Score 0.07373
# FOLD 3  (xgb 0.242) (xgb1 0.165) (xgb2 0.189) (xgb3 0.240) (xgb4 0.177) (xgb5 0.114) (xgb6 0.167) (xgb7 0.262) (xgb8 0.138) (lgb 0.182) (lgb1 0.117) (lgb2 0.126) (lgb3 0.211) (lgb4 0.192) (lgb5 0.063) 
==> Ensemble Score 0.08516
# FOLD 4  (xgb 0.268) (xgb1 0.169) (xgb2 0.160) (xgb3 0.130) (xgb4 0.139) (xgb5 0.140) (xgb6 0.179) (xgb7 0.132) (xgb8 0.253) (lgb 0.165) (lgb1 0.243) (lgb2 0.249) (lgb3 0.211) (lgb4 0.197) (lgb5 0.160) 
==> Ensemble Score 0.12184
# FOLD 5  (xgb 0.437) (xgb1 0.201) (xgb2 0.200) (xgb3 0

## Stacking Model

In [12]:
%%time

stack_test_predss = np.zeros((X_test.shape[0]))
stack_scores = []
stack_models = []

splitter = Splitter(kfold = kfold, n_splits=n_splits, greeks = greeks.iloc[:,1:-1])
for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(oof_each_predss, y_train, random_state_list=random_state_list)):
    n = i % n_splits
    m = i // n_splits
    
#     model =  SVC(C=5, probability=True)
    model = LogisticRegression(solver='liblinear')
    model.fit(X_train_ , y_train_)
    
    
    y_val_m_pred = model.predict_proba(X_val)[:, 1]
    test_pred = model.predict_proba(test_each_predss)[:, 1]
    
    score = balanced_log_loss(y_val, y_val_m_pred)
    print(f'--> Stacking [FOLD-{n+1} SEED-{random_state_list[m]}] BalancedLogLoss score {score:.5f}')
    if n ==0: print()
    stack_scores.append(score)
    stack_models.append(deepcopy(model))
    stack_test_predss += test_pred / (n_splits * len(random_state_list))
# Calculate the mean LogLoss score of the ensemble
mean_score = np.mean(stack_scores)
std_score = np.std(stack_scores)

--> Stacking [FOLD-1 SEED-1824] BalancedLogLoss score 0.20231

--> Stacking [FOLD-2 SEED-1824] BalancedLogLoss score 0.11910
--> Stacking [FOLD-3 SEED-1824] BalancedLogLoss score 0.15131
--> Stacking [FOLD-4 SEED-1824] BalancedLogLoss score 0.15711
--> Stacking [FOLD-5 SEED-1824] BalancedLogLoss score 0.17488
--> Stacking [FOLD-6 SEED-1824] BalancedLogLoss score 0.19094
--> Stacking [FOLD-7 SEED-1824] BalancedLogLoss score 0.11936
--> Stacking [FOLD-8 SEED-1824] BalancedLogLoss score 0.31556
--> Stacking [FOLD-9 SEED-1824] BalancedLogLoss score 0.09787
--> Stacking [FOLD-10 SEED-1824] BalancedLogLoss score 0.32088
--> Stacking [FOLD-1 SEED-409] BalancedLogLoss score 0.21146

--> Stacking [FOLD-2 SEED-409] BalancedLogLoss score 0.11590
--> Stacking [FOLD-3 SEED-409] BalancedLogLoss score 0.14135
--> Stacking [FOLD-4 SEED-409] BalancedLogLoss score 0.25416
--> Stacking [FOLD-5 SEED-409] BalancedLogLoss score 0.21190
--> Stacking [FOLD-6 SEED-409] BalancedLogLoss score 0.23664
--> Stackin

## OverAll Score


In [13]:
# Calculate the mean LogLoss score of the ensemble
mean_score = np.mean(ensemble_score)
std_score = np.std(ensemble_score)
print(f'Ensemble BalancedLogLoss score {mean_score:.5f} ± {std_score:.5f}')
# Print the mean and standard deviation of the ensemble weights for each model
print('--- Model Weights ---')
mean_weights = np.mean(weights, axis=0)
std_weights = np.std(weights, axis=0)
for name, mean_weight, std_weight in zip(models.keys(), mean_weights, std_weights):
    print(f'{name}: {mean_weight:.5f} ± {std_weight:.5f}')
print('')

# Calculate the mean LogLoss score of the ensemble
mean_score = np.mean(stack_scores)
std_score = np.std(stack_scores)
print(f'Stacking BalancedLogLoss score {mean_score:.5f} ± {std_score:.5f}\n')


Ensemble BalancedLogLoss score 0.16684 ± 0.08113
--- Model Weights ---
xgb: 0.04988 ± 0.17481
xgb1: 0.06282 ± 0.18225
xgb2: 0.03042 ± 0.12287
xgb3: 0.19142 ± 0.33345
xgb4: 0.09910 ± 0.24107
xgb5: 0.29355 ± 0.38691
xgb6: 0.06850 ± 0.19074
xgb7: 0.22605 ± 0.34891
xgb8: 0.18574 ± 0.32789
lgb: 0.01922 ± 0.08344
lgb1: 0.18785 ± 0.34407
lgb2: 0.12981 ± 0.27848
lgb3: 0.37969 ± 0.40891
lgb4: 0.33180 ± 0.40811
lgb5: 0.33981 ± 0.41874

Stacking BalancedLogLoss score 0.18182 ± 0.07075



## Scores for each model

In [14]:
# def plot_score_from_dict(score_dict, title='', ascending=True):
    
#     score_df = pd.melt(pd.DataFrame(score_dict))
    
#     score_df = score_df.sort_values('value', ascending=ascending)
    
#     plt.figure(figsize=(14, 8))
#     sns.barplot(x='value', y='variable', data=score_df, palette='Blues_r', errorbar='sd')
#     plt.xlabel(f'{title}', fontsize=14)
#     plt.ylabel('')
#     #plt.title(f'{title}', fontsize=18)
#     plt.xticks(fontsize=12)
#     plt.yticks(fontsize=12)
#     plt.grid(True, axis='x')
#     plt.show()


# weight_dict = dict(zip(list(score_dict.keys()), np.array(weights).T.tolist()))
# plot_score_from_dict(weight_dict, title='Model Weights', ascending=False)
# normalize = [((weight - np.min(weight)) / (np.max(weight) - np.min(weight))).tolist() for weight in weights]
# weight_dict = dict(zip(list(score_dict.keys()), np.array(normalize).T.tolist()))
# plot_score_from_dict(weight_dict, title='Model Weights (Normalize 0 to 1)', ascending=False)

In [15]:
# plot_score_from_dict(score_dict, title=f'BalancedLogLoss (n_splits:{n_splits})')

In [16]:
# plot_score_from_dict(weight_dict, title='Model Weights (Normalize 0 to 1)', ascending=False)

## PREDICT MODEL

In [17]:
PREDICT_1 = test_predss
PREDICT_2 = stack_test_predss

In [18]:
# preds = stack_test_predss
# preds = test_predss * 0.50 + stack_test_predss * 0.50


In [19]:
# sub = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'))
# sub['class_1'] = preds
# sub['class_0'] = 1 - preds
# sub.to_csv('submission.csv', index=False)
# sub

# **NOTEBOOK NUMBER 2**

In [20]:
!pip install -q /kaggle/input/tabpfn-019-whl/tabpfn-0.1.9-py3-none-any.whl
!mkdir /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/tabpfn-019-whl/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [21]:
import pandas as pd, numpy as np
pd.set_option('display.max_columns', 100)
import matplotlib.pyplot as plt
import pickle

import cudf
from cuml.svm import SVC

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
import itertools

from catboost import CatBoostClassifier, CatBoostRegressor,  Pool
import category_encoders as ce

from tabpfn import TabPFNClassifier
from sklearn.manifold import TSNE

import warnings
warnings.filterwarnings('ignore')

In [22]:
def pond_logloss(ytrue, preds):
    preds = np.clip(preds, 1e-15, 1-1e-15)
    n0 = (ytrue==0).sum()
    n1 = (ytrue==1).sum()
    
    v0 = -np.sum(np.log(1-preds[ytrue==0]))/2/n0
    v1 = -np.sum(np.log(preds[ytrue==1]))/2/n1
    
    return v1+v0

def lgb_metric(y_true, y_pred):
    return 'balanced_log_loss', pond_logloss(y_true, y_pred), False

# https://www.kaggle.com/code/datafan07/icr-simple-eda-baseline
def balance_logloss(y_true, y_pred):
    
    y_pred = np.stack([1-y_pred,y_pred]).T
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    y_pred / np.sum(y_pred, axis=1)[:, None]
    nc = np.bincount(y_true)
    
    logloss = (-1/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(y_pred[:,0]))) - 1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred[:,1])))) / 2
    
    return logloss


In [23]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

df = pd.concat([train,test],axis=0)
print('data shape', train.shape )
df.head()

data shape (617, 58)


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1.0
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0.0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0.0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0.0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1.0


In [24]:
# tree based feature
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
train['EJ_A'] = (train['EJ'] =='A').astype(int)
train['EJ_B'] = (train['EJ'] =='B').astype(int)
# isna feature
train['BQ_isna']  = train.BQ.isna().astype(int)
train['EL_isna']  = train.EL.isna().astype(int)

train.drop('EJ',axis=1,inplace=True)

FEATURES = []
for c in train.columns:
    if c in ['Id','Class']:continue
    FEATURES.append(c)
    
print(f'I will train {len(FEATURES)} Feature')

I will train 59 Feature


In [25]:
xgb_params_1 ={ 'colsample_bylevel' : 0.6, 'colsample_bytree' : 1.0, 
'learning_rate' : 0.3,  'max_depth' : 4,  'n_estimators' : 1000,  'subsample' : 1.0,    'scale_pos_weight' : 5}

cat_params_1 = {"depth":4, "l2_leaf_reg" : 50, "learning_rate" : 0.3,
"objective": "Logloss", "logging_level":"Silent", "iterations" : 10000,
"auto_class_weights" : "Balanced", "one_hot_max_size":10}

lgbm_params = {
    "max_depth": 4,
    "num_leaves": 9,
    "min_child_samples": 17,
    "n_estimators": 200,
    "learning_rate": 0.15,
    "colsample_bytree": 0.4,
    "min_split_gain": 1e-4,
    "reg_alpha": 1e-2,
    "reg_lambda": 5e-3,
    'early_stopping_round':10,
}
xgb_params_2 = {
        'objective': 'binary:logistic',
        'learning_rate': 0.05, 
     'n_estimators' : 1000,
        'max_depth': 3,
        'colsample_bytree': 0.50,
        'subsample': 0.80,
        'eta': 0.03,
        'gamma': 1.5,
#         'tree_method':'gpu_hist'
    }
    
cat_params_2 = {
        'learning_rate': 0.05, 
        'iterations': 10000, 
        'depth': 3, # 
        'colsample_bylevel': 0.50,
        'subsample': 0.80,
        'l2_leaf_reg': 3, # 3, 30
        'auto_class_weights': 'Balanced',
#         'task_type':'GPU', 
    }

In [26]:
BAGS = 20
FOLDS = 10
## try XGBoost but not work

oof_lgb = np.zeros(len(train))
oof_xgb = np.zeros(len(train))
oof_cbt = np.zeros(len(train))

oof_xgb_2 = np.zeros(len(train))
oof_cbt_2 = np.zeros(len(train))

# model saved
models_lgb = {}
models_xgb = {}
models_cbt = {}

models_xgb_2 = {}
models_cbt_2 = {}
for bag in range(BAGS):
    
    print('### Bag',bag+1)
    
    models_lgb[bag] = []
    models_xgb[bag] =[]
    models_cbt[bag] =[]
    
    models_xgb_2[bag] =[]
    models_cbt_2[bag] =[]
    
    skf = KFold(n_splits=FOLDS, shuffle=True, random_state=bag)
    for fold,(train_idx, valid_idx) in enumerate(skf.split(X=train[FEATURES], y=train['Class'])):
    
        print('=> Fold',fold+1,' ',end='')
        
        # DOWNSAMPLE NEGATIVE CLASS TO BALANCE CLASSES
        y_train = train.loc[train_idx,'Class']
        RMV = y_train.loc[y_train==0].sample(
            frac=0.7, random_state=bag*BAGS+fold).index.values
        train_idx = np.setdiff1d(train_idx,RMV)

        # TRAIN DATA
        X_train = train.loc[train_idx,FEATURES]
        y_train = train.loc[train_idx,'Class']
        # VALID DATA
        X_valid = train.loc[valid_idx,FEATURES]
        y_valid = train.loc[valid_idx,'Class']
        
        eval_set=[(X_train, y_train), (X_valid, y_valid)]
        
        # LGBM 
        clf = LGBMClassifier(**lgbm_params)
        clf.fit(X_train, y_train, eval_metric = 'logloss', eval_set = (X_valid, y_valid),  verbose=0)
        print(f'({clf.best_iteration_})' , end=' ')
        oof_lgb[valid_idx] += clf.predict_proba(X_valid)[:,1] / BAGS
        models_lgb[bag].append(clf)
        
        # XGB 1
        clf = XGBClassifier(**xgb_params_1)
        clf.fit(X_train, y_train, eval_metric = ["logloss"], eval_set = eval_set, early_stopping_rounds = 10, verbose=0)
        print(f'({clf.best_iteration})' , end=' ')
        oof_xgb[valid_idx] += clf.predict_proba(X_valid)[:,1] / BAGS
        models_xgb[bag].append(clf)
        
        # CBT 1
        clf = CatBoostClassifier(**cat_params_1)
        clf.fit(X_train,y_train ,eval_set = eval_set, verbose=0,early_stopping_rounds=10)
        print(f'({clf.best_iteration_})' , end=' ')
        oof_cbt[valid_idx] += clf.predict_proba(X_valid)[:,1] / BAGS
        models_cbt[bag].append(clf)
        
        # XGB 2
        clf = XGBClassifier(**xgb_params_2)
        clf.fit(X_train, y_train, eval_metric = ["logloss"], eval_set = eval_set, early_stopping_rounds = 10, verbose=0)
        print(f'({clf.best_iteration})' , end=' ')
        oof_xgb_2[valid_idx] += clf.predict_proba(X_valid)[:,1] / BAGS
        models_xgb_2[bag].append(clf)
        
        # CBT 2
        clf = CatBoostClassifier(**cat_params_2)
        clf.fit(X_train,y_train ,eval_set = [(X_train, y_train), (X_valid, y_valid)], verbose=0,early_stopping_rounds=10)
        print(f'({clf.best_iteration_})' , end=' ,')
        oof_cbt_2[valid_idx] += clf.predict_proba(X_valid)[:,1] / BAGS
        models_cbt_2[bag].append(clf)
        
    print()
    
print('-'*150)
print('CV Score : ')
# CV Score : 
oof_tpc = pd.read_csv('/kaggle/input/tabpfn-model/oof_tpc.csv')
oof_tpc = oof_tpc.values.squeeze()
mt = balance_logloss( train.Class.values, oof_tpc )
print('TabPFN = ',mt.round(3))
mg = balance_logloss( train.Class.values, oof_lgb )
print('LGBM   = ',mg.round(3))
mx = balance_logloss( train.Class.values, oof_xgb )
print('XGB    = ',mx.round(3))
mc = balance_logloss( train.Class.values, oof_cbt )
print('CBT    = ',mc.round(3))

mx2 = balance_logloss( train.Class.values, oof_xgb_2 )
print('XGB_2  = ',mx2.round(3))
mc2 = balance_logloss( train.Class.values, oof_cbt_2 )
print('CBT_2  = ',mc2.round(3))

### Bag 1
=> Fold 1  (19) (4) (25) (70) (83) ,=> Fold 2  (122) (55) (145) (152) (242) ,=> Fold 3  (53) (56) (99) (210) (198) ,=> Fold 4  (33) (18) (56) (134) (109) ,=> Fold 5  (52) (17) (74) (167) (182) ,=> Fold 6  (60) (26) (131) (166) (173) ,=> Fold 7  (54) (47) (42) (101) (139) ,=> Fold 8  (52) (43) (85) (117) (171) ,=> Fold 9  (37) (45) (97) (120) (196) ,=> Fold 10  (53) (21) (87) (107) (199) ,
### Bag 2
=> Fold 1  (48) (24) (25) (65) (175) ,=> Fold 2  (75) (151) (119) (184) (209) ,=> Fold 3  (40) (12) (23) (119) (112) ,=> Fold 4  (42) (46) (47) (191) (122) ,=> Fold 5  (57) (20) (47) (168) (205) ,=> Fold 6  (38) (39) (75) (127) (229) ,=> Fold 7  (46) (31) (76) (106) (91) ,=> Fold 8  (53) (21) (46) (77) (197) ,=> Fold 9  (43) (10) (69) (113) (147) ,=> Fold 10  (38) (41) (85) (95) (129) ,
### Bag 3
=> Fold 1  (22) (7) (19) (67) (92) ,=> Fold 2  (88) (39) (143) (161) (205) ,=> Fold 3  (39) (42) (109) (182) (210) ,=> Fold 4  (31) (9) (33) (121) (130) ,=> Fold 5  (81) (64) (184) (201) (

In [27]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
train['EJ_A'] = (train['EJ'] =='A').astype(int)
train['EJ_B'] = (train['EJ'] =='B').astype(int)

# isna feature
train['BQ_isna']  = train.BQ.isna().astype(int)
train['EL_isna']  = train.EL.isna().astype(int)

train.drop('EJ',axis=1,inplace=True)

# SAVE MEANS, STDS, NANS FOR TEST INFERENCE LATER
means = {}; stds = {}; nans = {}
for c in train.columns:
    if c in ['Id','Class']:continue
    m = train[c].mean()
    means[c] = m
    s = train[c].std()
    stds[c] = s
    train[c] = (train[c]-m)/s
    n = train[c].min() - 0.5
    nans[c] = n
    train[c] = train[c].fillna(n)

In [28]:
BAGS = 20
FOLDS = 10
oof_svc = np.zeros(len(train))
oof_lr = np.zeros(len(train))

models_svc = {}
models_lr = {}

for bag in range(BAGS):
#     print('#'*25)
    print('### Bag',bag+1)
#     print('#'*25)
    models_svc[bag] = []
    models_lr[bag] = []
    
    skf = KFold(n_splits=FOLDS, shuffle=True, random_state=bag)
    for fold,(train_idx, valid_idx) in enumerate(skf.split(X=train[FEATURES], y=train['Class'])):

        print('=> Fold',fold+1,', ',end='')

        # DOWNSAMPLE NEGATIVE CLASS TO BALANCE CLASSES
        y_train = train.loc[train_idx,'Class']
        RMV = y_train.loc[y_train==0].sample(
            frac=0.7, random_state=bag*BAGS+fold).index.values
        train_idx = np.setdiff1d(train_idx,RMV)

        # TRAIN DATA
        X_train = train.loc[train_idx,FEATURES]
        y_train = train.loc[train_idx,'Class']

        # VALID DATA
        X_valid = train.loc[valid_idx,FEATURES]
        y_valid = train.loc[valid_idx,'Class']

        
        # SVC
        clf = SVC(C=5, probability=True) 
        clf.fit(X_train, y_train)
        oof_svc[valid_idx] += clf.predict_proba(X_valid).iloc[:,1].values / BAGS
        models_svc[bag].append(clf)
        
        #Logistic Regression
        clf = LogisticRegression(solver='liblinear')
        clf.fit(X_train, y_train)
        oof_lr[valid_idx] += clf.predict_proba(X_valid)[:,1] / BAGS
        models_lr[bag].append(clf)
        
    print()
    
ml = balance_logloss( train.Class.values, oof_lr )
print('LogisticRegression score = ',ml.round(3))
ms = balance_logloss( train.Class.values, oof_svc )
print('SVM score = ',ms.round(3))

### Bag 1
=> Fold 1 , => Fold 2 , => Fold 3 , => Fold 4 , => Fold 5 , => Fold 6 , => Fold 7 , => Fold 8 , => Fold 9 , => Fold 10 , 
### Bag 2
=> Fold 1 , => Fold 2 , => Fold 3 , => Fold 4 , => Fold 5 , => Fold 6 , => Fold 7 , => Fold 8 , => Fold 9 , => Fold 10 , 
### Bag 3
=> Fold 1 , => Fold 2 , => Fold 3 , => Fold 4 , => Fold 5 , => Fold 6 , => Fold 7 , => Fold 8 , => Fold 9 , => Fold 10 , 
### Bag 4
=> Fold 1 , => Fold 2 , => Fold 3 , => Fold 4 , => Fold 5 , => Fold 6 , => Fold 7 , => Fold 8 , => Fold 9 , => Fold 10 , 
### Bag 5
=> Fold 1 , => Fold 2 , => Fold 3 , => Fold 4 , => Fold 5 , => Fold 6 , => Fold 7 , => Fold 8 , => Fold 9 , => Fold 10 , 
### Bag 6
=> Fold 1 , => Fold 2 , => Fold 3 , => Fold 4 , => Fold 5 , => Fold 6 , => Fold 7 , => Fold 8 , => Fold 9 , => Fold 10 , 
### Bag 7
=> Fold 1 , => Fold 2 , => Fold 3 , => Fold 4 , => Fold 5 , => Fold 6 , => Fold 7 , => Fold 8 , => Fold 9 , => Fold 10 , 
### Bag 8
=> Fold 1 , => Fold 2 , => Fold 3 , => Fold 4 , => Fold 5 , => Fol

In [29]:
m0 = balance_logloss( train.Class.values, oof_lgb )
print('LGBM score = ',m0.round(3))
m6 = balance_logloss( train.Class.values, oof_tpc )
print('TPFN score = ',m6.round(3))
m4 = balance_logloss( train.Class.values, oof_xgb )
print('XGB score = ',m4.round(3))
m5 = balance_logloss( train.Class.values, oof_cbt )
print('CBT score = ',m5.round(3))
m1 =  balance_logloss( train.Class.values, oof_svc )
print('SVC score = ',m1.round(3))
m2 =  balance_logloss( train.Class.values, oof_lr )
print('LR  score = ',m2.round(3))
m8 =  balance_logloss( train.Class.values, oof_xgb_2 )
print('XGB2score = ',m8.round(3))
m8 =  balance_logloss( train.Class.values, oof_cbt_2 )
print('CBT2 score = ',m8.round(3))
## I dorp SVM from ensemble
oof = \
(oof_tpc * 0.20) + \
    (oof_lgb * 0.20) + \
        (oof_xgb * 0.25) + \
            (oof_cbt * 0.20) + \
                (oof_lr  * 0.05) + \
                    (oof_xgb_2  * 0.05) + \
                        (oof_cbt_2  * 0.05)
m = balance_logloss( train.Class.values, oof )
print('*'*40)
print('CV Score =',m.round(3))

LGBM score =  0.221
TPFN score =  0.278
XGB score =  0.219
CBT score =  0.248
SVC score =  0.34
LR  score =  0.327
XGB2score =  0.279
CBT2 score =  0.249
****************************************
CV Score = 0.23


In [30]:
print('TabPFN Model predict..')
models_tpc = {}
for bag in range(1,6):
    models_tpc[bag] =[]
    for i in range(1,6):
        loaded_model = pickle.load(open(f'/kaggle/input/tabpfn-model/bag_{bag}_fold_{i}.sav', 'rb'))
        models_tpc[bag].append(loaded_model)
        

test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
test['EJ_A'] = (test['EJ'] =='A').astype(int)
test['EJ_B'] = (test['EJ'] =='B').astype(int)
# isna feature
test['BQ_isna']  = test.BQ.isna().astype(int)
test['EL_isna']  = test.EL.isna().astype(int)
test.drop('EJ',axis=1,inplace=True)

preds_tpc = np.zeros(len(test))

for bag in range(1,6):
    for clf in models_tpc[bag]:
            X_test = test[FEATURES]
            preds_tpc += clf.predict_proba(X_test)[:,1] / 5 / 5

TabPFN Model predict..


In [31]:
print('# None-Tree Based Model Predict..')
preds_svc = np.zeros(len(test))
preds_lr = np.zeros(len(test))

test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
test['EJ_A'] = (test['EJ'] =='A').astype(int)
test['EJ_B'] = (test['EJ'] =='B').astype(int)

# isna feature
test['BQ_isna']  = test.BQ.isna().astype(int)
test['EL_isna']  = test.EL.isna().astype(int)

test.drop('EJ',axis=1,inplace=True)

for c in test.columns:
    if c in ['Id','Class']:continue
    m = means[c]
    s = stds[c]
    test[c] = (test[c]-m)/s
    n = nans[c]
    test[c] = test[c].fillna(n)
    
for bag in range(BAGS):
    for clf in models_svc[bag]:
            X_test = test[FEATURES]
            preds_svc += clf.predict_proba(X_test).iloc[:,1].values / FOLDS / BAGS
            
    for clf in models_lr[bag]:
            X_test = test[FEATURES]
            preds_lr += clf.predict_proba(X_test)[:,1] / FOLDS / BAGS

# None-Tree Based Model Predict..


In [32]:
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
test['EJ_A'] = (test['EJ'] =='A').astype(int)
test['EJ_B'] = (test['EJ'] =='B').astype(int)
# isna feature
test['BQ_isna']  = test.BQ.isna().astype(int)
test['EL_isna']  = test.EL.isna().astype(int)

test.drop('EJ',axis=1,inplace=True)

print('Models Predict (LGBM)..')
preds_lgb = np.zeros(len(test))
for bag in range(BAGS):
    for clf in models_lgb[bag]:
            X_test = test[FEATURES]
            preds_lgb += clf.predict_proba(X_test)[:,1] / FOLDS / BAGS
            
print('Models Predict (CBT)..')
preds_cbt = np.zeros(len(test))
for bag in range(BAGS):
    
    for clf in models_cbt[bag]:
            X_test = test[FEATURES]
            preds_cbt += clf.predict_proba(X_test)[:,1] / FOLDS / BAGS
            
print('Models Predict (XGB)..')
preds_xgb = np.zeros(len(test))
for bag in range(BAGS):
    
    for clf in models_xgb[bag]:
            X_test = test[FEATURES]
                        
            preds_xgb += clf.predict_proba(X_test)[:,1] / FOLDS / BAGS
            
            
print('Models Predict (CB_2)..')
preds_cbt_2 = np.zeros(len(test))
for bag in range(BAGS):
    
    for clf in models_cbt_2[bag]:
            X_test = test[FEATURES]
                        
            preds_cbt_2 += clf.predict_proba(X_test)[:,1] / FOLDS / BAGS
            
print('Models Predict (XGB2)..')
preds_xgb_2 = np.zeros(len(test))
for bag in range(BAGS):
    
    for clf in models_xgb_2[bag]:
            X_test = test[FEATURES]
                        
            preds_xgb_2 += clf.predict_proba(X_test)[:,1] / FOLDS / BAGS

Models Predict (LGBM)..
Models Predict (CBT)..
Models Predict (XGB)..
Models Predict (CB_2)..
Models Predict (XGB2)..


In [33]:
print('Ensamble..')
preds = \
    preds_tpc * 0.20 + \
        preds_lgb * 0.20 + \
            preds_xgb * 0.25 + \
                preds_cbt * 0.20 + \
                    preds_lr * 0.05 + \
                        preds_xgb_2 * 0.05 + \
                            preds_cbt_2 * 0.05

Ensamble..


In [34]:
PREDICT_3 = preds

## **SUBMISSION**

In [35]:
FINAL_PREDICT = PREDICT_1 * 0.3 + PREDICT_2 * 0.4 + PREDICT_3 * 0.3

In [36]:
sub = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'))
sub['class_1'] = FINAL_PREDICT
sub['class_0'] = 1 - FINAL_PREDICT
sub.to_csv('submission.csv', index=False)
sub

,Id,class_0,class_1
0,00eed32682bb,0.716184,0.283816
1,010ebe33f668,0.716184,0.283816
2,02fa521e1838,0.716184,0.283816
3,040e15f562a2,0.716184,0.283816
4,046e85c7cc7f,0.716184,0.283816
